# Extracting Information from Citizenship(BACK)

In [74]:
#imporing libraries
import cv2 #opencv helps to work with image
import pytesseract
import re

In [95]:
#read image
im = cv2.imread("citizenav.png")

In [96]:
#color image to grey
def make_grey(image):
    new_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2.imwrite("img_gray.png",new_img)
    return new_img

In [97]:
#greyscaled image to blurry
def make_blurry(image):
    new_img = cv2.GaussianBlur(image, (5,5),0)
    cv2.imwrite("img_blurry.png", new_img)
    return new_img

In [98]:
def set_threshold(image):
    #pixels with value below 50 are turned black and those with higher value are turned white (255)
    new_img = cv2.threshold(image, 50, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)[1] 
    cv2.imwrite("img_threshold.png",new_img)
    return new_img

In [99]:
im_gray = make_grey(im)
im_blur = make_blurry(im_gray)
im_thresh = set_threshold(im_blur)

In [100]:
contours, hierarchy = cv2.findContours(im_thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [101]:
def contours_text(orig, contours):
    for cnt in contours: 
        x, y, w, h = cv2.boundingRect(cnt) 

        # Drawing a rectangle on copied image 
        rect = cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 255, 255), 2) 
        
        # Cropping the text block for giving input to OCR 
        cropped = orig[y:y + h, x:x + w] 

        # Apply OCR on the cropped image 
        config = ('-l eng --oem 1 --psm 6')
        text = pytesseract.image_to_data(cropped, output_type=pytesseract.Output.DICT, config=config) 
                
        return text

In [102]:
data = contours_text(im, contours)

In [103]:
parse_text = []
word_list = []

last_word = ''

for word in data['text']:
    if word!='':
        word_list.append(word)
        last_word = word

    if (last_word!='' and word == '') or (word==data['text'][-1]):
        parse_text.append(word_list)
        word_list = []

In [104]:
parse_text

[['|',
  'Goverment',
  'of',
  'Nepal',
  'has',
  'sued',
  'this',
  'Citizenship',
  'Cerificate',
  'with',
  'folowing',
  'detais,'],
 ['Citizenship', 'Certificate', 'No.', '$0-01-71-03087', 'Sex:', 'Mak'],
 ['|', 'Full', 'Name.', 'ABHISHEK', 'SAPKOTA'],
 ['Date', 'of', 'Birth', '(AD):', 'Year-i997', 'MontaNOV', '—Day:07'],
 ['Birth', 'Place:', 'District:', 'Baghing'],
 ['Municipality.', 'Baghnne', 'Ward', 'So.4'],
 ['Permanent', 'Address-', 'District', 'Baglung'],
 ['Municipatity', 'Raghing', 'Ward', 'No.4'],
 ['ara', 'Cet', '2otg', 'Eaton', 'amie', 'Warrer', 'Sl'],
 ['amiftarer', 'Rattan', 're'],
 ['SANT', 'Tr', 'SIRT', 'eT', 'se'],
 ['wT', 'cal)'],
 ['are', 'At', '4', 'oe'],
 ['2', 'oe', 'saror', 'oem', 'RA', '(0'],
 ['ai', 'area', ':'],
 ['£', '‘', 'aa', 'ot', 'BET', 'neal', '|'],
 ['|', 'a', '|', 'RSS', 'ee,'],
 ['4', 'ls', 'aro', 'in', 'sealord', 'ag'],
 ['at',
  'WOM',
  'RE',
  'Coat',
  'aT',
  'Teter',
  'erica',
  'at',
  'eet',
  'acchrearaempfegiey!']]

In [105]:
#changing all lines with list to single string
line1 = ' '.join([str(elem) for elem in parse_text[1]])
line2 = ' '.join([str(elem) for elem in parse_text[2]])
line3 = ' '.join([str(elem) for elem in parse_text[3]])

In [106]:
#pass line1 as line1 has all info related to citizen num
def find_citizenship_num(line):
    citizenship_num = re.findall('[0-9]+', line)
    citizenship_num = '-'.join([str(elem) for elem in citizenship_num])
    return citizenship_num

In [107]:
#pass line2 as line 2 has all info related to name
def find_name(line):
    regex_for_name = r"\b[A-Z][A-Z]+\b"
    full_name = re.findall(regex_for_name, line)
    full_name = ' '.join([str(elem) for elem in full_name])
    return full_name

In [108]:
#pass line3 as line3 has all date info
def find_date(line):
    '''regex to find dates and month'''
    date_regex = r'\d+'
    dates = re.findall(date_regex, line)
    month_regex = r'\b[A-Z]{3}'
    month = re.findall(month_regex, line)
    dates.append(month[0])
    dates = '-'.join(str(elem) for elem in dates)
    return dates

In [109]:
citizenship_num = find_citizenship_num(line1)

In [110]:
sex = parse_text[1][-1]

In [111]:
full_name = find_name(line2)

In [112]:
dates = find_date(line3)

IndexError: list index out of range

In [113]:
retreived = {}
retreived['name'] = full_name
retreived['citizenship_number'] = citizenship_num
retreived['sex'] = sex
retreived['dob'] = dates

In [114]:
retreived

{'name': 'ABHISHEK SAPKOTA',
 'citizenship_number': '0-01-71-03087',
 'sex': 'Mak',
 'dob': '2001-14-APR'}